### Import mix module 

In [11]:
import numpy as np
import pandas as pd
from CODECbreakCode.AudioMixer import SingleFileAudioMixer
import CODECbreakCode.Evaluator as Evaluator
from CODECbreakCode.Evaluator import MeasureHAAQIOutput
import CODECbreakCode.NoiseEval as NEUtil
from Optimiser.config import get_config, normalize_action, denormalize_action
import argparse

In [2]:
import warnings
warnings.filterwarnings("ignore", "Possible clipped samples in output.")
warnings.filterwarnings("ignore",message="Warning: input samples dtype is np.float64. Converting to np.float32")

In [3]:
Mixing_Path = '/home/codecrack/CodecBreakerwithRL/AudioEX/'
Noise_Generator_MP3 = SingleFileAudioMixer(filename="reggae.wav", foldpath=Mixing_Path, TrackType = NEUtil.MixingType.File)
Referece_File,_ = Noise_Generator_MP3.TestNoisedOnlyFile([0,0,0,0],"Reference.wav")
Referece_File
Referece_MP3File = Evaluator.Mp3LameLossyCompress(Referece_File,64)
print(f"Referece_MP3File:{Referece_MP3File}")

Audio duration orginal is 17.462666666666667 seconds, now is the 8.0, the audio changing to the MONO
Referece_MP3File:/home/codecrack/CodecBreakerwithRL/AudioEX/Mixing_Result_Mp3_Wav/Reference_64kbps.wav


In [4]:
####initalise the Haaqi
MeasureHAAQI = MeasureHAAQIOutput(Referece_MP3File)#Initilize the HAAQI with a permanent reference
MeasureHAAQI.MeasureHAQQIOutput(Referece_MP3File) #Test on how far from itself to itself

0.998548685133253

In [ ]:
denormalize_action(np.array([-3.604144,0.5862619,-1.3705522,-1.853153]))

In [5]:
def haaqi_reward_fn(solution: np.ndarray, is_normalised = True) -> float:
    if is_normalised:
        solution = denormalize_action(solution)
#    print(f'solution:{solution}')
    thre, ratio, attack, release = solution
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
#    print(f'solution:{solution} with reward {round(1-score,4)}')
    return round(1-score,4)

In [15]:
###inital
haaqi_reward_fn(np.array([-30,   10,   1, 104.3]),is_normalised = False)

0.2509

In [ ]:
import numpy as np
m1=np.array([-0.5,-0.5,-0.5,-0.5])
m2=np.array([0.5,0.5,0.5,0.5])

def f(x):
    return -np.sum(np.log(np.sum((x-m1)**2)+0.00001)-np.log(np.sum((x-m2)**2)+0.01))

## Ranking

### function

In [ ]:
######ranking corresspoding funciton 
def haaqi_reward_fn(solution: np.ndarray) -> float:
    thre = solution[0]
    thre = thre - 15
    ratio = solution[1]
    ratio = ratio + 5
    attack = solution[2]
    attack = attack + 10
    release = solution[3] 
    release = release + 300
    degradated_filename = f'audio_mixing_reggea_dynC_thre_{thre}_ratio_{ratio}_attk_{attack}_rele_{release}.wav'
    gener_Audio = Noise_Generator_MP3.TestNoisedOnlyFileOnlyDynamicCompressor(outputfilename = degradated_filename,
                                                               threshold = thre,
                                                               ratio = ratio,
                                                               attack = attack,
                                                               release = release)
    gener_Audio_mp3 = Evaluator.Mp3LameLossyCompress(gener_Audio,64)
    score = MeasureHAAQI.MeasureHAQQIOutput(gener_Audio_mp3)
#    print(f"Current degraded thre:{thre}, ratio: {ratio}, attack:{attack},release:{release} bring the HAAQI score to {score}")
    return 1-score

### GA

In [ ]:
###Worst result for GA Until the 11.July
###haaqi_reward_fn(np.array([-12.4, 2.7, -6.8, -56]))
####Worst result for GA Until the 12.July 
haaqi_reward_fn(np.array([-14.9, 3.3, -9.0, -193]))

### RL in Continous

In [ ]:
####Worst result for GA Until the 12.July [-1.2019364  -5.0801964  -3.631278  -54.986313]
haaqi_reward_fn(np.array([-1.2019,-5.0801,-3.6313,-54.9863]))

### File Clear

In [10]:
Noise_Generator_MP3.EraseTheMp3Mixing()

All files and folders in '/home/codecrack/CodecBreakerwithRL/AudioEX//Mixing_Result' have been removed.


### Import Discrete RL module

In [ ]:
import Optimiser

In [ ]:
import Optimiser.discrete_RL_train
from Optimiser.discrete_RL_train import discrete_RL_train as RLTrain

In [ ]:
trainner = RLTrain()
trainner.set_environments(haaqi_reward_fn)
trainner.train()

In [ ]:
###Worst result for discrete ReinforcementL Until the 11.July
haaqi_reward_fn(np.array([ -3.9 ,  -0.5 ,  -4.4 , -52.72]))

### Import Continous RL

In [31]:
import Optimiser

In [32]:
import Optimiser.continous_RL_train
from Optimiser.continous_RL_train import continous_RL_train as CRLTrain

In [33]:
trainner = CRLTrain(sub_episode_length=6, sub_episode_num_single_batch=5, env_num=1)
trainner.set_environments(haaqi_reward_fn)
trainner.train(update_num=3, eval_intv=1)

number of sub_episodes used for a single param update: 5
train_env.batch_size = parallel environment number =  1
update_num: 3, eval_intv: 1
[step   0] mean entropy per-dim = 5.6065
final reward before udpate: -1000000000.0
final reward after udpate: 0.2077
updated final_solution= [-30.    10.     8.42 100.  ]
train_step no.= 1
best_solution of this generation= [-30.    10.     8.42 100.  ]
best step reward= 0.2077
avg step reward= 0.06655601
final_solution= [-30.    10.     8.42 100.  ] final_reward= 0.2077
act_std: [-3.9000e-01  9.8800e+00  1.9750e+01  4.9482e+02]
act_mean: [-15.    5.5  10.5 300. ]
obs_mean: [-11.43   7.32  10.37 264.2 ]
obs_std: [  0.  10.  20. 500.]
best_step_index: [4, 3]
 
[step   1] mean entropy per-dim = 5.5726
final reward before udpate: 0.2077
final reward after udpate: 0.2207
updated final_solution= [-30.    10.     1.   251.06]
train_step no.= 2
best_solution of this generation= [-30.    10.     1.   251.06]
best step reward= 0.2207
avg step reward= 0.1050

In [ ]:
RL_data, RL_F_data = trainner.save_results(filefold=Mixing_Path, para_columns=['Thre','Ratio','Attk','Release'],is_outputfulldata = False)

In [ ]:
final_reward_list

In [ ]:
REINFORCE_logs

In [41]:
haaqi_reward_fn(np.array([-15,5.5,10.5,300]),is_normalised=False)

solution:[-15.    5.5  10.5 300. ] with reward 0.11


0.11

### Import GA module

In [ ]:
import Optimiser.genetic_evo
from Optimiser.genetic_evo import GeneticOptimiser

In [ ]:
GA_Opt = GeneticOptimiser()

In [ ]:
GA_Opt.ga_init_env()

In [ ]:
GA_Opt.set_fitnessfun(haaqi_reward_fn)

In [ ]:
GA_Opt.run()

In [ ]:
Evo_Data, Evo_Data_Full = GA_Opt.save_results(filefold=Mixing_Path, genre_columns=['Thre','Ratio','Attk','Release'], is_outputfulldata = True)

In [ ]:
Evo_Data

In [ ]:
Evo_Data_Full

### Import MC Optimiser

In [ ]:
import Optimiser.mc_opt
from Optimiser.mc_opt import MonteCarloOptimiser

In [ ]:
mc = MonteCarloOptimiser(reward_fn = haaqi_reward_fn)
per_step_bests = mc.optimise_by_steps(n_steps=5, sampling_per_step=3)